In [2]:
conda install datasets

Retrieving notices: done
Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/genai_env

  added / updated specs:
    - datasets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    anaconda-2025.04           | py310_openblas_0          72 KB
    anyio-4.6.2                |  py310hca03da5_0         206 KB
    bcrypt-3.2.0               |  py310h80987f9_2          37 KB
    certifi-2025.1.31          |  py310hca03da5_0         163 KB
    datasets-3.3.2             |  py310hca03da5_0         654 KB
    fsspec-2024.12.0           |  py310hca03da5_0         302 KB
    h11-0.14.0                 |  py310hca03da5_0          92 KB
    httpcore-1.0.2             |  py310hca03da5_0          87 KB
    httpx-0.27.0               |  py310hca03da5_0         173 KB
    huggingface_hub-0.31.2     |  py310hca03da5_0         55

In [3]:
import pandas as pd
from datasets import load_dataset

In [4]:
from datasets import load_dataset

ds = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")

README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


part.0.parquet:   0%|          | 0.00/54.4k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/918 [00:00<?, ? examples/s]